<a href="https://colab.research.google.com/github/cutefishaep/BRender/blob/main/BRender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 BRender v1.0
**BRender** is a lightweight, streamlined Google Colab notebook designed to leverage cloud GPU resources for rendering Blender projects. It supports seamless `.blend` file uploads, archive extraction, and customizable rendering parameters for both animations and single frames.

## 🚀 Key Features

* **Versatile Version Support:** Supports multiple Blender versions ranging from `2.79b` to the newest version.
* **Archive Support:** Upload a single `.blend` file or a `.zip` archive containing assets and textures.
* **Smart File Detection:** Automatically detects `.blend` files within uploaded zips. If multiple files are found, an interactive menu allows you to select the correct target.
* **GPU Acceleration:** Built-in support for CUDA and OptiX rendering pipelines.
* **Automated Packaging:** Automatically zips output frames for easy downloading.

## 📋 How to Use

1.  **Configure Parameters**
    Before executing the cell, adjust the settings in the "Configuration Parameters" section:
    * `blender_version`: Choose your target Blender version.
    * `animation`: Check this box to render a sequence; uncheck for a single frame.
    * `start_frame` / `end_frame`: Define the rendering range.
    * `output_name`: Set the filename prefix (e.g., `render-##`).
    * `gpu_enabled`: Ensure this is checked to utilize Colab's GPU.

2.  **Run the Script**
    Execute the main code cell. The script will automatically install dependencies (including `libtcmalloc` for memory optimization) and download the selected Blender version.

3.  **Upload Project**
    When prompted, upload your file:
    * **`.blend`**: For simple scenes with packed resources.
    * **`.zip`**: For complex scenes with external texture folders/assets.

4.  **Select Scene (If applicable)**
    If your uploaded zip contains multiple `.blend` files, the notebook will pause and ask you to input the number corresponding to the file you wish to render.

5.  **Download Results**
    Once rendering is complete, the output (image or zip archive) will be automatically downloaded to your local machine.

## ⚙️ Configuration Details

| Parameter | Type | Description |
| :--- | :--- | :--- |
| `optix_enabled` | Boolean | Enables OptiX rendering (faster raytracing). *Note: Automatically disabled for older GPUs like Tesla K80.* |
| `zip_files` | Boolean | If enabled, all rendered frames are compressed into a single zip file before download. |
| `cpu_enabled` | Boolean | Fallback option to use CPU (not recommended for heavy scenes). |

## 📝 Credits
Developed and maintained by **@cutefishrbx** on TikTok.

In [ ]:
# @title **Configuration**
# Import necessary libraries
import os
import shutil
import requests
from google.colab import files
from IPython.display import clear_output

# --- Configuration Parameters (from original cell 8dFNjYGTgNjR) ---
blender_version = '4.5.4' #@param ['2.79b', '2.83.20', '2.93.18', '3.3.21', '3.6.23', '4.2.14', '4.5.3'] {allow-input: true}
animation = True #@param {type: 'boolean'}
start_frame =  0#@param {type: 'integer'}
end_frame =  67#@param {type: 'integer'}
output_name = 'blender-##' #@param {type: 'string'}
zip_files = True #@param {type: 'boolean'}
gpu_enabled = True #@param {type:"boolean"}
optix_enabled = False #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}
debug = False #@param {type:"boolean"}
auto_disconnect = False #@param {type:"boolean"}

# --- Helper function for clearing output ---
def clear_if_not_debug():
    if not debug:
        clear_output(wait=True)

# --- Initial Setup and GPU Check (from original cells XQ0cynSzSYDd) ---
print("Changing directory to /content")
os.chdir('/content')

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

if gpu[0] == "Tesla K80" and optix_enabled:
  print("OptiX disabled because of unsupported GPU")
  optix_enabled = False

# --- Install libtcmalloc-minimal4 (from original cell pQboAtzI0Zc8) ---
clear_if_not_debug()
print("\n--- Installing libtcmalloc-minimal4 ---")
os.environ["LD_PRELOAD"] = ""
!apt remove libtcmalloc-minimal4 -y
!apt install libtcmalloc-minimal4 -y
os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.5.9"
print("libtcmalloc-minimal4 installed.")

# --- File Upload (from original cell QRzNmiHN8Xr0) ---
clear_if_not_debug()
uploaded_filename = ""
print("\n--- Uploading File ---")
uploaded = files.upload()
for fn in uploaded.keys():
    uploaded_filename = fn
print(f"Uploaded file: {uploaded_filename}")

# --- Process Uploaded File (from original cell KowqWtLzgSLw) ---
clear_if_not_debug()
print("\n--- Processing Uploaded File ---")
!rm -rf render
!mkdir render

if uploaded_filename.lower().endswith('.zip'):
    !unzip -o $uploaded_filename -d 'render/'
elif uploaded_filename.lower().endswith('.blend'):
    shutil.copy(uploaded_filename, 'render/')
else:
    raise SystemExit("Invalid file extension, only .blend and .zip can be uploaded.")
print("File processed.")

# --- Auto Detect .blend Files ---
clear_if_not_debug()
print("\n--- Detecting .blend Files ---")

def find_blend_files(directory):
    """Recursively find all .blend files in directory"""
    blend_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.blend'):
                # Get relative path from render directory
                full_path = os.path.join(root, file)
                relative_path = os.path.relpath(full_path, directory)
                blend_files.append(relative_path)
    return blend_files

blend_files = find_blend_files('/content/render')

if len(blend_files) == 0:
    raise SystemExit("No .blend files found in the uploaded file!")
elif len(blend_files) == 1:
    blend_file_path = blend_files[0]
    print(f"✓ Found 1 .blend file: {blend_file_path}")
else:
    print(f"\n{'='*50}")
    print("More than 1 .blend file detected!")
    print(f"{'='*50}\n")

    for idx, file in enumerate(blend_files, 1):
        print(f"{idx}. {file}")

    print(f"\n{'='*50}")

    # Get user input
    while True:
        try:
            choice = input("\nChoose which one do you want to render (enter number): ")
            choice_num = int(choice)

            if 1 <= choice_num <= len(blend_files):
                blend_file_path = blend_files[choice_num - 1]
                print(f"\n✓ Selected: {blend_file_path}")
                break
            else:
                print(f"❌ Please enter a number between 1 and {len(blend_files)}")
        except ValueError:
            print("❌ Please enter a valid number")
        except KeyboardInterrupt:
            raise SystemExit("\nRender cancelled by user.")

print(f"\nBlend file to render: {blend_file_path}")

# --- Download and Extract Blender (from original cell GPJ9fmoB6PWM) ---
clear_if_not_debug()
print("\n--- Downloading Blender ---")
blender_url_dict = {
    '2.79b'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2",
    '2.83.20' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.83/blender-2.83.20-linux-x64.tar.xz"
}

if blender_version in blender_url_dict:
    blender_url = blender_url_dict[blender_version]
else:
    major_minor = ".".join(blender_version.split('.')[:2])
    blender_url = f"https://ftp.nluug.nl/pub/graphics/blender/release/Blender{major_minor}/blender-{blender_version}-linux-x64.tar.xz"

try:
    response = requests.head(blender_url, allow_redirects=True, timeout=10)
    if response.status_code != 200:
        print(f"Download failed for version '{blender_version}'.")
        print("Error downloading: You may need to define the download archive manually above.")
    else:
        base_url = os.path.basename(blender_url)
        print(f"Download URL: {blender_url}")
        print(f"Base filename: {base_url}")
except Exception as e:
    print(f"Error checking URL: {e}")
    print("Error downloading: You may need to define the download archive manually above.")

!mkdir -p $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1
print("Blender downloaded and extracted.")

# --- Set up GPU Configuration (from original cell 6elYiQrvkUAe) ---
clear_if_not_debug()
print("\n--- Setting up GPU Configuration ---")
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

renderer = "CUDA"
if optix_enabled:
    print("Note: You're currently using OptiX renderer. If an error occurred, the current GPU (e.g. Tesla K80) is not supported and you need to switch back to CUDA.")
    renderer = "OPTIX"
print("GPU configuration script created.")

# --- Start Blender Render (from original cell Fpk2w3yM8XqQ) ---
clear_if_not_debug()
print("\n--- Starting Blender Render ---")
os.chdir('/content') # Ensure we are in /content before creating output directory

!rm -rf output
!mkdir output

output_path = '/content/output/' + output_name

os.chdir(f'/content/{blender_version}') # Change to Blender directory for execution

if animation:
    if start_frame == end_frame:
        !./blender -b '/content/render/{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -a -- --cycles-device "{renderer}"
    else:
        !./blender -b '/content/render/{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -s $start_frame -e $end_frame -a -- --cycles-device "{renderer}"
else:
    !./blender -b '/content/render/{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -f $start_frame -- --cycles-device "{renderer}"
print("Blender rendering complete.")

# --- Download Rendered Files (from original cell IjN_CFwEIB6f) ---
clear_if_not_debug()
print("\n--- Downloading Rendered Files ---")
os.chdir('/content') # Ensure we are back in /content

path, dirs, files_folder = next(os.walk("output"))
output_folder_name = output_name.replace('#', '') + 'render'

if len(files_folder) == 1:
    render_img = 'output/' + files_folder[0]
    files.download('output/' + files_folder[0])
elif len(files_folder) > 1:
    if zip_files:
        shutil.make_archive(output_folder_name, 'zip', 'output')
        files.download(output_folder_name + '.zip')
    else:
        for f in files_folder:
            files.download('output/{}'.format(f))
elif not files_folder:
    raise SystemExit("No frames are rendered.")
print("Rendered files downloaded.")

# --- Auto Disconnect Runtime (if enabled) ---
if auto_disconnect:
    clear_if_not_debug()
    print("\n--- Auto Disconnect Enabled ---")
    print("Disconnecting Google Colab runtime...")
    from google.colab import runtime
    runtime.unassign()

In [ ]:
# @title (OPTIONAL) Render Only (EXPERINMENTAL)

import os
import shutil
from google.colab import files

# --- Set up GPU Configuration ---
print("\n--- Setting up GPU Configuration ---")
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

renderer = "CUDA"
if optix_enabled:
    print("Note: Using OptiX renderer.")
    renderer = "OPTIX"
print("GPU configuration script created.")

# --- Start Blender Render ---
print("\n--- Starting Blender Render ---")
os.chdir('/content')

!rm -rf output
!mkdir output

output_path = '/content/output/' + output_name

os.chdir(f'/content/{blender_version}')

if animation:
    if start_frame == end_frame:
        !./blender -b '/content/render/{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -a -- --cycles-device "{renderer}"
    else:
        !./blender -b '/content/render/{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -s $start_frame -e $end_frame -a -- --cycles-device "{renderer}"
else:
    !./blender -b '/content/render/{blend_file_path}' -P "/content/setgpu.py" -E CYCLES -o '{output_path}' -noaudio -f $start_frame -- --cycles-device "{renderer}"
print("Blender rendering complete.")

# --- Download Rendered Files ---
print("\n--- Downloading Rendered Files ---")
os.chdir('/content')

path, dirs, files_folder = next(os.walk("output"))
output_folder_name = output_name.replace('#', '') + 'render'

if len(files_folder) == 1:
    render_img = 'output/' + files_folder[0]
    files.download('output/' + files_folder[0])
elif len(files_folder) > 1:
    if zip_files:
        shutil.make_archive(output_folder_name, 'zip', 'output')
        files.download(output_folder_name + '.zip')
    else:
        for f in files_folder:
            files.download('output/{}'.format(f))
elif not files_folder:
    raise SystemExit("No frames are rendered.")
print("Rendered files downloaded.")